In [ ]:
## Run this command for GPU

#from tensorflow.compat.v1 import ConfigProto
#from tensorflow.compat.v1 import InteractiveSession

#config = ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
#config.gpu_options.allow_growth = True
#session = InteractiveSession(config=config)

In [1]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Check TensorFlow version
tf.__version__

'2.3.1'

In [3]:
# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [4]:
training_set = train_datagen.flow_from_directory('datasets/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('datasets/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8029 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
from tensorflow.keras.layers import Conv2D

In [7]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn_model = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn_model.add(tf.keras.layers.Conv2D(filters=32, padding="same", kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn_model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn_model.add(tf.keras.layers.Conv2D(filters=32, padding='same', kernel_size=3, activation='relu'))
cnn_model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn_model.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn_model.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [8]:
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [9]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
cnn_model.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
251/251 [==============================] - 72s 285ms/step - loss: 0.6638 - accuracy: 0.5906 - val_loss: 0.6162 - val_accuracy: 0.6420
Epoch 2/10
251/251 [==============================] - 74s 293ms/step - loss: 0.5809 - accuracy: 0.6936 - val_loss: 0.5552 - val_accuracy: 0.7320
Epoch 3/10
251/251 [==============================] - 77s 307ms/step - loss: 0.5347 - accuracy: 0.7341 - val_loss: 0.5130 - val_accuracy: 0.7540
Epoch 4/10
251/251 [==============================] - 72s 287ms/step - loss: 0.5076 - accuracy: 0.7428 - val_loss: 0.4935 - val_accuracy: 0.7535
Epoch 5/10
251/251 [==============================] - 68s 270ms/step - loss: 0.4881 - accuracy: 0.7600 - val_loss: 0.4772 - val_accuracy: 0.7740
Epoch 6/10
251/251 [==============================] - 67s 268ms/step - loss: 0.4625 - accuracy: 0.7832 - val_loss: 0.4738 - val_accuracy: 0.7710
Epoch 7/10
251/251 [==============================] - 68s 271ms/step - loss: 0.4460 - accuracy: 0.7910 - val_loss: 0.4757 - val_ac

In [10]:
# save it as a h5 file

from tensorflow.keras.models import load_model

cnn_model.save('model_rcat_dog.h5')

In [11]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('model_rcat_dog.h5')

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [13]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('datasets/single_prediction/cat_or_dog_1.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = cnn_model.predict(test_image)

In [14]:
result

array([[0.99996424]], dtype=float32)

In [15]:
if result[0]<=0.5:
    print("The image classified is cat")
else:
    print("The image classified is dog")

The image classified is dog


In [ ]:
# For multi class problems few changes are needed

# Step 5 - Output Layer: Change units based on class and activation function as softmax
#cnn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compiling the CNN: Change loss function as categorical_crossentropy
# cnn_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])